# MobileNetV3-rpi with SSD

In [1]:
from model import MobileNetV3_RPiWide, BlockNoDw, hswish
import torch
from torch import nn
from torch.nn import Conv2d, Sequential, ModuleList, BatchNorm2d
import torch.nn.functional as F
import config
from torchinfo import summary
from ssd import SSD

In [2]:
# NOTE: input size is 480, 288 (1, 3, 288, 480)

width_mult = 1.0
num_classes = 2

model = MobileNetV3_RPiWide(width_mult=width_mult, num_classes=num_classes).eval()
base_net = model.features
base_net

Sequential(
  (0): Conv2d(3, 8, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): hswish()
  (3): BlockNoDw(
    (se): SeModule(
      (se): Sequential(
        (0): AdaptiveAvgPool2d(output_size=1)
        (1): Conv2d(8, 2, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (3): ReLU(inplace=True)
        (4): Conv2d(2, 8, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (5): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (6): hsigmoid()
      )
    )
    (conv1): Conv2d(8, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (nolinear1): ReLU(inplace=True)
    (conv2): Conv2d(16, 8, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    

In [3]:
out = base_net(torch.randn(1, 3, 300, 300))
out.shape

torch.Size([1, 288, 10, 10])

In [4]:
source_layer_indexes = [ 11, 17 ]
extras = ModuleList([
    BlockNoDw(3, 288, 256, 512, hswish(), None, stride=2),
    BlockNoDw(3, 512, 128, 256, hswish(), None, stride=2),
    BlockNoDw(3, 256, 128, 256, hswish(), None, stride=2),
    BlockNoDw(3, 256, 64, 64, hswish(), None, stride=2)
])

regression_headers = ModuleList([
    Conv2d(in_channels=round(24 * width_mult), out_channels=6 * 4, kernel_size=3, padding=1),
    Conv2d(in_channels=288, out_channels=6 * 4, kernel_size=3, padding=1),
    Conv2d(in_channels=512, out_channels=6 * 4, kernel_size=3, padding=1),
    Conv2d(in_channels=256, out_channels=6 * 4, kernel_size=3, padding=1),
    Conv2d(in_channels=256, out_channels=6 * 4, kernel_size=3, padding=1),
    Conv2d(in_channels=64, out_channels=6 * 4, kernel_size=1),
])

classification_headers = ModuleList([
    Conv2d(in_channels=round(24 * width_mult), out_channels=6 * num_classes, kernel_size=3, padding=1),
    Conv2d(in_channels=288, out_channels=6 * num_classes, kernel_size=3, padding=1),
    Conv2d(in_channels=512, out_channels=6 * num_classes, kernel_size=3, padding=1),
    Conv2d(in_channels=256, out_channels=6 * num_classes, kernel_size=3, padding=1),
    Conv2d(in_channels=256, out_channels=6 * num_classes, kernel_size=3, padding=1),
    Conv2d(in_channels=64, out_channels=6 * num_classes, kernel_size=1),
])

ssd_model = SSD(num_classes, base_net, source_layer_indexes, extras, classification_headers, regression_headers, is_test=False, config=config)
ssd_model.eval()

SSD(
  (base_net): Sequential(
    (0): Conv2d(3, 8, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): hswish()
    (3): BlockNoDw(
      (se): SeModule(
        (se): Sequential(
          (0): AdaptiveAvgPool2d(output_size=1)
          (1): Conv2d(8, 2, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (2): BatchNorm2d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (3): ReLU(inplace=True)
          (4): Conv2d(2, 8, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (5): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (6): hsigmoid()
        )
      )
      (conv1): Conv2d(8, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (nolinear1): ReLU(inplace=True)
      (conv2): Conv2d(16, 8, kernel_siz

In [5]:
summary(ssd_model, (1, 3, 300, 300), device='cpu')

Layer (type:depth-idx)                             Output Shape              Param #
SSD                                                --                        --
├─ModuleList: 1-1                                  --                        --
├─ModuleList: 1-2                                  --                        --
├─ModuleList: 1-3                                  --                        --
├─ModuleList: 1-4                                  --                        --
├─Sequential: 1                                    --                        --
│    └─Conv2d: 2-1                                 [1, 8, 150, 150]          216
│    └─BatchNorm2d: 2-2                            [1, 8, 150, 150]          16
│    └─hswish: 2-3                                 [1, 8, 150, 150]          --
│    └─BlockNoDw: 2-4                              [1, 8, 75, 75]            --
│    │    └─Conv2d: 3-1                            [1, 16, 150, 150]         128
│    │    └─BatchNorm2d: 3-2     

In [6]:
out = ssd_model(torch.randn(1, 3, 300, 300))
out[0].shape, out[1].shape

(torch.Size([1, 3000, 2]), torch.Size([1, 3000, 4]))

In [8]:
print(torch.__version__)

t_model = torch.jit.trace(model, torch.randn(1, 3, 300, 300))
torch.jit.save(t_model, 'mb3-rpi-ssd.torchscript')

1.8.0+cpu


C:\Development\anaconda3\envs\lpcv\lib\site-packages\torch\_jit_internal.py:603: LightningDeprecationWarning: The `LightningModule.datamodule` property is deprecated in v1.3 and will be removed in v1.5. Access the datamodule through using `self.trainer.datamodule` instead.
  if hasattr(mod, name):
C:\Development\anaconda3\envs\lpcv\lib\site-packages\torch\_jit_internal.py:603: LightningDeprecationWarning: The `LightningModule.loaded_optimizer_states_dict` property is deprecated in v1.4 and will be removed in v1.6.
  if hasattr(mod, name):
